In [4]:
import pandas as pd
import math
import random
import geopandas as gpd
from shapely.geometry import Point, Polygon


def haversine(lat1: float, lon1: float, lat2: float, lon2: float) -> float:
    R = 6371000
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)

    a = (
        math.sin(dphi / 2) ** 2
        + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2) ** 2
    )
    
    d = 2 * R * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return d

# --- Загрузка данных ---
df = pd.read_csv("dataset/chipotle_stores.csv")
states = gpd.read_file("dataset/us-states.json")

# --- Фильтрация данных ---
ny_polygon = states[states["id"] == "NY"].geometry.values[0]
nyc_df = df[df["state"].str.contains("New York", case=False, na=False)]


# --- Функция для генерации случайной точки в пределах Нью-Йорка ---
def random_point_in_polygon(polygon: Polygon) -> Point:
    minx, miny, maxx, maxy = polygon.bounds
    while True:
        p = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        if polygon.contains(p):
            return p


# --- Функция для симуляции ---
def simulate(n_points=1000):
    counts = {"A (пешком)": 0, "B (самокат)": 0, "C (троллейбус)": 0}

    min_dists = []

    for _ in range(n_points):
        point = random_point_in_polygon(ny_polygon)
        lat_rand, lon_rand = point.y, point.x

        # расстояние до ближайшего Chipotle
        distances = nyc_df.apply(
            lambda row: haversine(
                lat_rand, lon_rand, row["latitude"], row["longitude"]
            ),
            axis=1,
        )
        min_dist = distances.min()
        min_dists.append(min_dist)

        # выбор транспорта
        if min_dist <= 1000:
            counts["A (пешком)"] += 1
        elif min_dist < 10000:
            counts["B (самокат)"] += 1
        else:
            counts["C (троллейбус)"] += 1

    print("Среднее расстояние до ближайшего Chipotle:", sum(min_dists) / n_points)
    print("Медиана расстояния до ближайшего Chipotle:", sorted(min_dists)[n_points // 2])

    # вероятность
    probs = {k: v / n_points for k, v in counts.items()}
    return counts, probs


counts, probs = simulate(5000)

print("Частоты:", counts)
print("Вероятности:", probs)

Среднее расстояние до ближайшего Chipotle: 43101.64815000326
Медиана расстояния до ближайшего Chipotle: 38763.50635791181
Частоты: {'A (пешком)': 19, 'B (самокат)': 448, 'C (троллейбус)': 4533}
Вероятности: {'A (пешком)': 0.0038, 'B (самокат)': 0.0896, 'C (троллейбус)': 0.9066}
